<img src="./images/DLI_Header.png" style="width: 400px;">

# 卷積神經網路

在上一節中，我們建立並訓練了簡單的模型來分類美國手語影像。模型能夠學會如何以極高的準確度正確分類訓練資料集，但在驗證資料集上卻沒有達到相同水準的表現。這種無法順利歸納非訓練資料的行為稱為[過度擬合](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html)，在本節，我們會介紹一種常見的模型[卷積神經網路](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53)，特別適合用於讀取和分類影像。

## 目標

* 為卷積神經網路準備專用資料
* 建立更精密的卷積神經網路模型，瞭解更多不同的模型層
* 訓練卷積神經網路模型並觀察效能

## 載入和準備資料

下方的儲存格包含我們在前一次實驗室中學到的資料前置處理技術。請先複習並執行，再繼續操作：

In [ ]:
import tensorflow.keras as keras
import pandas as pd

# Load in our data from CSV files
train_df = pd.read_csv("asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

## 為卷積神經網路重新調整影像

在上一個練習中，資料集內個別圖片的格式是 784 像素的陣列：

In [ ]:
x_train.shape, x_valid.shape

在這種格式中，我們無法得知哪些像素彼此相鄰的完整資訊。因此，我們無法應用卷積來偵測特徵。讓我們重新調整資料集，改為 28x28 像素的格式。這樣一來，我們的卷積就能將在相關聯的像素上偵測重要特徵。

請注意，對於模型的第一個卷積層來說，我們不僅需要影像的高度和寬度，還需要[色頻](https://www.photoshopessentials.com/essentials/rgb/)的數量。我們的影像是灰階，因此我們只有 1 個色頻。

這表示我們需要將目前的形狀 `(27455, 784)`轉換成 `(27455, 28, 28, 1)`。為方便起見，我們可以傳遞給 [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape) 方法 `-1`，表示任何我們想要保持不變的維度，因此：

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [ ]:
x_train.shape

In [ ]:
x_valid.shape

In [ ]:
x_train.shape, x_valid.shape

## 建立卷積模型

現在，許多資料科學家在展開專案時，都會從類似專案中借用模型。假設問題並非完全獨一無二，就有很高的機率，可以透過其他人效能良好的模型，獲取相近的效能，舉例這些線上模型儲存庫如 [TensorFlow Hub](https://www.tensorflow.org/hub)和 [NGC](https://ngc.nvidia.com/catalog/models)。今天，我們要提供一種相當適合解決這個問題的模型。

<img src="images/cnn.png" width=180 />

我們在課程中介紹了許多不同類型的模型層，在這裡我們會重新回顧一次並提供相關的說明文件連結。如果有疑問，請閱讀官方說明文件 (或詢問 [stackoverflow](https://stackoverflow.com/))。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))

### [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

<img src="images/conv2d.png" width=300 />

這些是我們的 2D 卷積層。小型核心(過濾器)會遍歷輸入影像，並偵測分類所需的重要特徵。模型中較前段的卷積會偵測到簡單的特徵，例如線條。後段的卷積則會偵測到更複雜的特徵。來看看我們的第一個 Conv2D 層：
```Python
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same'...)
```
75 指的是將要學習的核心數量。(3,3) 指的是這些核心的大小。strides指的是濾鏡經過影像時採用的步長。padding指的是從濾鏡建立的輸出影像是否與輸入影像的大小相符。

### [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization)

和正規化輸入一樣，BatchNormalization(批次正規化)會調整隱藏層中的值，以提升訓練品質。[在此閱讀更多詳細資訊](https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c)。

### [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

<img src="images/maxpool2d.png" width=300 />
MaxPool2D(最大池化)會將影像縮小成較低的解析度。這麼做是為了協助模型能夠在影像平移時保有泛化能力，也有助於提升模型的速度。

### [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)

<img src="images/dropout.png" width=360 />
Dropout 是預防過度擬合的技術。Dropout 會隨機選取並關閉神經元的子集，所以這些神經元不會在特定階段中參與正向或反向傳播。這有助於確保網路的泛化和降低冗餘神經元發生機率，且不依賴任何單一區域來產生答案。

### [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)

Flatten 會將某一層的多維輸出，扁平化成為一維陣列。這種輸出稱為特徵向量，將會連結至最終分類層。

### [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)

我們已經在先前的模型中看到密集層。我們的第一個密集層 (512 單位) 會把特徵向量視為輸入，並學習哪些特徵將會影響到特定分類。第二個密集層 (24 單位) 是最終的分類層，負責輸出預測結果。

## 總結模型

你可能會覺得資訊實在太多，但別擔心。即使你無法立即瞭解所有內容，也能有效訓練卷積模型。最重要的是，你明白這類模型可以協助你從影像中擷取出有用的資訊，而且可以用於分類工作。

我們來總結一下剛剛建立的模型：

In [ ]:
model.summary()

## 編寫模型

我們會像先前一樣編寫模型：

In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

## 訓練模型

儘管模型架構非常不同，訓練流程基本上完全相同。執行下方儲存格以進行 20 個 Epoch 的訓練，讓我們看看準確度是否有所改善：

In [ ]:
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid))

## 結果討論

看來模型已經有顯著提升了！訓練準確度很高，而且驗證準確度也有所改善。這個結果很理想，因為我們只需要換一種新模型即可。

你可能已經注意到，驗證準確度會有變動。這表示我們的模型仍沒有完美泛化的能力。所幸在這方面我們還有其他方法。我們會在下節課程談到這部分。

## 摘要

在本節中，你利用了幾種新的模型層來實作卷積神經網路，這比上一節使用的簡化模型表現更好。希望你已經更加熟悉整體流程，能夠運用準備好的資料來建立和訓練模型。

## 清除記憶體
在繼續之前，請執行下列儲存格以清除 GPU 記憶體。必須先完成此步驟才能繼續進行下一個 Notebook。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

在前幾節中，你的學習重點是建立和訓練模型。為了進一步提升模型效能，現在你要將重點擺在*資料增強*，這一系列的技術可讓模型在更多且更合適的資料上進行訓練，更勝你原本可運用的資料。

請繼續前往下一節：[*資料增強*](./04a_asl_augmentation.ipynb).